In [1]:
import os; print(os.getcwd())
import socket; print(socket.gethostname())
import os
from glob import glob
from Bio import SeqIO
import truncator as u
import pathlib

/home/aljubetic/AF2/projects/2023-03-21__mALB8_all_by_all_v2
headnode


In [2]:
seqs = SeqIO.to_dict(SeqIO.parse(open('data/mALB8.fasta'),'fasta'))
out_dir = pathlib.Path('out/01__multimer-v2')

In [3]:

#make a big all-by-all fasta file


with open('01-all-by-all.fasta', 'w') as out_file:
    for id1 in seqs.keys():
        for id2 in seqs.keys():
            name = f'{id1}___{id2}'
            # get rid of - and white space
            seq1 = ''.join(str(seqs[id1].seq).replace('-','').split())
            seq2 = ''.join(str(seqs[id2].seq).replace('-','').split())
            out_file.write(f'>{name}\n')
            out_file.write(f'{seq1}:{seq2}\n')
                

In [4]:
def write_to_fasta(name, list_of_seq):
    with open(name, 'w') as out_file:
        for seq in list_of_seq:
            out_file.write(f'>{seq.id}\n')
            out_file.write(f'{seq.seq}\n')

In [5]:
#load the big fasta file
seqs = SeqIO.to_dict(SeqIO.parse(open('01-all-by-all.fasta'),'fasta'))
#Sort by length
seqs = dict(sorted(seqs.items(), key=lambda item: len(item[1])))
write_to_fasta('01-all-by-all.fasta', seqs.values())


In [6]:
MAX_GROUP_SIZE =12
MAX_GROUP_TOTAL_AA   = 3000
MAX_SIZE_CHANGE = 1.1

GROUPS = []

seq_list = list(seqs.values())

#add fist sequence to group

seq = seq_list[0]
group_index = 0
GROUPS.append([seq])
group_size = 1
group_total_AA = len(seq) 
last_added_length = len(seq)

for seq in seq_list[1:]:
    #if change in criteria
    size_change = len(seq)/last_added_length  
    if group_size > MAX_GROUP_SIZE or group_total_AA > MAX_GROUP_TOTAL_AA or size_change > MAX_SIZE_CHANGE:
        #Make a new group
        group_index = group_index + 1
        GROUPS.append([seq])
        group_size = 1
        group_total_AA = len(seq) 
        last_added_length = len(seq)
    else:
        GROUPS[group_index].append(seq)
        group_size += 1
        group_total_AA += len(seq) 
        last_added_length = len(seq)



In [8]:
u.make_dirs(out_dir)
for n, g in enumerate(GROUPS):
    write_to_fasta(out_dir/f"g{n:04d}.fasta", g)

In [9]:
fastas = sorted(glob(f'{out_dir}/*.fasta'))

print(f'{out_dir}/run.tasks')
with open(f'{out_dir}/run.tasks', 'w') as f:
    for fasta in fastas:
        fasta_name = u.basename_noext(fasta)
        f.write(f'source /home/aljubetic/bin/set_up_AF2.3.sh && mkdir -p {out_dir/fasta_name} && '
        f'/home/aljubetic/AF2/CF2.3/colabfold-conda/bin/colabfold_batch  --msa-mode single_sequence  --num-recycle 6 --amber --model-type alphafold2_multimer_v2 {fasta} {out_dir/fasta_name} \n')


out/01__multimer-v2/run.tasks


In [10]:
cmds = u.read_file_lines(f'{out_dir}/run.tasks', trim=True)
cmds[0]

'source /home/aljubetic/bin/set_up_AF2.3.sh && mkdir -p out/01__multimer-v2/g0000 && /home/aljubetic/AF2/CF2.3/colabfold-conda/bin/colabfold_batch  --msa-mode single_sequence  --num-recycle 6 --amber --model-type alphafold2_multimer_v2 out/01__multimer-v2/g0000.fasta out/01__multimer-v2/g0000'

In [11]:
###TEST one
job_name='mALB8-all-by-all'
# -N 1 --nodelist=compute-3-[18,21],compute-6-0,compute-3-[19-20],compute-6-[1-4]
slurm_params = f'--partition=gpu --gres=gpu:A40:1 --ntasks=1 --cpus-per-task=2 --job-name={job_name} --output={job_name}.out -e {job_name}.err '

print(f"""sbatch  {slurm_params} --wrap="{cmds[0]}" """)

sbatch  --partition=gpu --gres=gpu:A40:1 --ntasks=1 --cpus-per-task=2 --job-name=mALB8-all-by-all --output=mALB8-all-by-all.out -e mALB8-all-by-all.err  --wrap="source /home/aljubetic/bin/set_up_AF2.3.sh && mkdir -p out/01__multimer-v2/g0000 && /home/aljubetic/AF2/CF2.3/colabfold-conda/bin/colabfold_batch  --msa-mode single_sequence  --num-recycle 6 --amber --model-type alphafold2_multimer_v2 out/01__multimer-v2/g0000.fasta out/01__multimer-v2/g0000" 


In [13]:
GROUP_SIZE=1
task_list=f'{out_dir}/run.tasks'
from math import ceil; num_tasks = ceil(len(cmds)/GROUP_SIZE)
!export GROUP_SIZE={GROUP_SIZE}; sbatch {slurm_params} -a 1-{num_tasks} /home/aljubetic/scripts/wrapper_slurm_array_job_group.sh {task_list} 

Submitted batch job 41628


In [16]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [38]:
!scancel 24290331 